## 프록시 디자인 패턴의 개요

프록시란 요청자와 공급자 사이의 중재자를 말한다.

디자인 패턴 관점에서 Proxy 클래스는 객체의 인터페이스 역할을 한다. 

객체는 네트워크 연결, 메모리, 파일에 저장된 객체 등 다양한 종류에 해당한다.

Proxy 클래스는 반환해 사용할 객체를 감싸는 포장지 또는 에이전트 객체다.

프록시 패턴의 주목적은 실제 객체에 접근할 수 있는 대리 객체나 껍데기를 제공하는 것이다.

프록시 패턴의 역할

* 복잡한 시스템을 간간하게 표현할 수 있다.
* 객체에 대한 보안을 제공한다.
* 일반적으로 클라이언트 객체에 직접 접근할 수 없다.
* 다른 서버에 존재하는 외부 객체에 대한 로컬 인터페이스를 제공한다. 분산 시스템 구조에서 클라이언트가 원격으로 특정 커맨트를 권한이 없어 수행하지 못하는 경우가 있다. 이런 경우 로컬 객체(프록시)에 요청을 보내고 프록시는 원격 서버에서 요청을 수행한다.
* 메모리 사용량이 높은 객체를 다루는 가벼운 핸들러 역할을 한다. 예를 들어 웹사이트 사용자의 프로필 사진은 목록에는 작은 이미지로 표시하고 세부 프로필에을 보여줄 때만 실제 사진을 로드하는 것이 효율적이다.

배우와 아이젠트의 관계를 통해 예제 구현

In [2]:
class Actor(object):
    def __init__(self):
        self.isBusy = False

    def occupied(self):
        self.isBusy = True
        print(type(self).__name__, '다른 영화 촬영 중')
    
    def available(self):
        self.isBusy = False
        print(type(self).__name__, '출연 가능')
    
    def getStatus(self):
        return self.isBusy

class Agent(object):
    def __init__(self):
        self.principal = None
    
    def work(self):
        self.actor = Actor()
        if self.actor.getStatus():
            self.actor.occupied()
        else:
            self.actor.available()
r = Agent()
r.work()

Actor 출연 가능


프록시 패턴의 역할은 다음과 같다.
    * 특정 객체의 대리 객체를 제공해 접근을 제어
    * 분산 구조를 지원하기 위한 레이어 또는 인터페이스를 제공
    * 의도하지 않은 케이스로부터 객체를 보호
    
## 프록시의 여러 유형들

### 가상 프록시

인스턴스화하기엔 무서운 객체의 플레이스홀더 역할을 한다. 웹에서 큰 이미지를 클릭했을 때 무거운 데이터를 불러오도록 한다.

### 원격 프록시

원격 서버나 다른 주소 공간에 존재하는 객체에 대한 로컬 인스턴스를 생성한다. 다수의 웹 서버와 데이터베이스 서버, 작업 서버, 캐시 서버 등으로 구성된 애플리케이션의 모니터링 시스템을 구성한다고 했을 때 각 서버의 CPU와 디스크 사용량을 모니터링하려면 모니터링 서버에서 각 서버의 실제 사용량 수치를 얻는 원격 명령을 수행할 수 있어야 한다. 

### 보호 프록시

근래의 분산 시스템에서 웹 애플리케이션은 여러 서비스를 조합해 기능을 제공한다. 이런 구조에서 사용자의 인증과 허가를 담당하는 인증 서비스가 보호 프록시 서버다. 허가 받지 않은 에이전트로부터 보호한다.

### 스마트 프록시

사용자가 객체에 접근했을 때 추가적인 행동을 취한다. 

## 프록시 패턴의 사용 사례

현금 카드의 사례

* 클라이언트 You 클래스
* make_payment() 메소드를 사용해 결제
* \__init__()는 프록시를 호출하고 생성하는 특수 메소드
* make_payment() 메소드를 내부적으로 Proxy의 메소드를 호출해 금액을 지불
* 결제가 성공하면 \__del__() 메소드를 호출

In [7]:
class You:
    def __init__(self):
        self.debitCard = DebitCard()
        self.isPurchased = None
    
    def make_payment(self):
        self.isPurchased = self.debitCard.do_pay()
    
    def __del__(self):
        if self.isPurchased:
            print("구매")
        else:
            print("돈이 부족")

In [8]:
from abc import ABCMeta, abstractmethod

class Payment(metaclass=ABCMeta):
    
    @abstractmethod
    def do_pay(self):
        pass

class Bank(Payment):
    def __init__(self):
        self.card = None
        self.account = None
    
    def _getAccount(self):
        self.account = self.card
        return self.account
    
    def _hasFunds(self):
        print("돈이 충분한지 확인")
        return True
    def setCard(self, card):
        self.card = card
    
    def do_pay(self):
        if self._hasFunds():
            print("지불")
            return True
        else:
            print("잔액이 부족")
            return False

class DebitCard(Payment):
    def __init__(self):
        self.bank = Bank()
    
    def do_pay(self):
        card = input("프록시 카드번호 입력")
        self.bank.setCard(card)
        return self.bank.do_pay()

In [9]:
you = You()
you.make_payment()

프록시 카드번호 입력123
돈이 충분한지 확인
지불


## 프록시 패턴의 장점

* 무거운 객체 특히 자주 사용되는 객체를 캐싱해 애플리케이션의 성능을 향상시킨다.
* RealSubject에 대한 접근 요청을 인증한다. 권한이 있을 때만 유효
* 원격 프록시는 원격 서버 간의 네트워크 연결과 데이터베이스 연결 구현에 적합하며 시스템 모니터링 용도로 사용될 수 있다.

## 퍼사드와 프록시 패턴 비교

* 프록시 패턴: 실 객체의 대리 객체를 재공해 접근 제어, 타켓 객체와 동일한 인터페이스 구조를 가지며 타켓에 대한 참조를 가지고 있다. 클라이언트와 감싸고 있는 객체 사이의 중재자 역할을 한다.

* 퍼사드 패턴: 클래스의 서브시스템에 대한 인터페이스를 제공, 서브시스템 간의 의존도와 통신을 최소화, 퍼사드 객체는 하나의 통합된 간단한 인터페이스를 제공